<a href="https://colab.research.google.com/github/primalbioinformatics/drug-design-2024/blob/main/QSAR_Regression_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas numpy scikit-learn rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 43.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_squared_error, r2_score
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

In [ ]:
# Load the dataset
data = pd.read_csv('acetylcholineestrase.csv')
# Example columns: 'SMILES', 'pIC50'
print(data.head())

                                              SMILES      pIC50
0  Cl.O=C(NCCCNc1c2c(nc3cc(Cl)ccc13)CCCC2)c1ccc2n...  11.221849
1  O=C(CCCCCCNc1c2c(nc3cc(Cl)cc(Cl)c13)CCCC2)NCCc...  11.096910
2  COc1ccccc1C(=O)N1Cc2cc(NC(=O)c3cccc(Cl)c3)ccc2...  10.960586
3  O=C(Nc1ccc2c(c1)CN(C(=O)c1ccccc1)C(=O)C2)c1ccc...  10.878440
4  Cc1cccc(C(=O)Nc2ccc3c(c2)CN(C(=O)c2cccc(C)c2)C...  10.869023


In [ ]:
# List all available descriptors in RDKit
descriptor_names = [desc_name[0] for desc_name in Descriptors.descList]

# Function to calculate all descriptors for a SMILES
def calculate_all_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)
    descriptors = calculator.CalcDescriptors(mol)
    return pd.Series(descriptors, index=descriptor_names)

# Apply the function to the dataset
descriptors = data['SMILES'].apply(calculate_all_descriptors)
data = pd.concat([data, descriptors], axis=1)
print(data.head())

Streaming output truncated to the last 5000 lines.
[09:34:41] DEPRECATION WARNING: please use MorganGenerator
[09:34:41] DEPRECATION WARNING: please use MorganGenerator
[09:34:41] DEPRECATION WARNING: please use MorganGenerator
[09:34:41] DEPRECATION WARNING: please use MorganGenerator
[09:34:41] DEPRECATION WARNING: please use MorganGenerator
[09:34:41] DEPRECATION WARNING: please use MorganGenerator
[09:34:41] DEPRECATION WARNING: please use MorganGenerator
[09:34:41] DEPRECATION WARNING: please use MorganGenerator
[09:34:41] DEPRECATION WARNING: please use MorganGenerator
[09:34:41] DEPRECATION WARNING: please use MorganGenerator
[09:34:41] DEPRECATION WARNING: please use MorganGenerator
[09:34:41] DEPRECATION WARNING: please use MorganGenerator
[09:34:41] DEPRECATION WARNING: please use MorganGenerator
[09:34:41] DEPRECATION WARNING: please use MorganGenerator
[09:34:41] DEPRECATION WARNING: please use MorganGenerator
[09:34:41] DEPRECATION WARNING: please use MorganGenerator
[09:3

                                              SMILES      pIC50  \
0  Cl.O=C(NCCCNc1c2c(nc3cc(Cl)ccc13)CCCC2)c1ccc2n...  11.221849   
1  O=C(CCCCCCNc1c2c(nc3cc(Cl)cc(Cl)c13)CCCC2)NCCc...  11.096910   
2  COc1ccccc1C(=O)N1Cc2cc(NC(=O)c3cccc(Cl)c3)ccc2...  10.960586   
3  O=C(Nc1ccc2c(c1)CN(C(=O)c1ccccc1)C(=O)C2)c1ccc...  10.878440   
4  Cc1cccc(C(=O)Nc2ccc3c(c2)CN(C(=O)c2cccc(C)c2)C...  10.869023   

   MaxAbsEStateIndex  MaxEStateIndex  MinAbsEStateIndex  MinEStateIndex  \
0          13.202586       13.202586           0.000000       -0.074427   
1          12.266648       12.266648           0.139956        0.139956   
2          13.022129       13.022129           0.109863       -0.415309   
3          12.722407       12.722407           0.149233       -0.324430   
4          12.887121       12.887121           0.165670       -0.302269   

        qed        SPS    MolWt  HeavyAtomMolWt  ...  fr_sulfide  \
0  0.154334  13.863636  647.050         612.778  ...         0.0   
1  0.17244

In [ ]:
# Fill missing values with the median of each descriptor
#data = data.fillna(data.median())

# Alternatively, drop descriptors with missing values
data = data.dropna(axis=1)


In [ ]:
# Define features (X) and target (y)
# Get the common columns between descriptor_names and data
available_descriptors = list(set(descriptor_names).intersection(data.columns))
X = data[available_descriptors]
y = data['pIC50']

In [ ]:
# Define features (X) and target (y)
# Get the common columns between descriptor_names and data
available_descriptors = list(set(descriptor_names).intersection(data.columns))
X = data[available_descriptors]
y = data['pIC50']

# Select top k features based on F-value
k = 5  # Adjust k according to your needs
selector = SelectKBest(score_func=f_regression, k=k)
X_selected = selector.fit_transform(X, y)

# Get the names of selected features
# Use available_descriptors instead of descriptor_names for indexing
selected_features = np.array(available_descriptors)[selector.get_support()]
print(f'Selected Features: {selected_features}')


Selected Features: ['EState_VSA5' 'VSA_EState7' 'SMR_VSA10' 'PEOE_VSA7' 'SlogP_VSA7']


In [ ]:
# Split the data into training and test sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize the model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)


In [ ]:
# Calculate R^2 and MSE for training and test sets
r2_train = r2_score(y_train, y_train_pred)
mse_train = mean_squared_error(y_train, y_train_pred)

r2_test = r2_score(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print(f'Training R^2: {r2_train:.2f}')
print(f'Training MSE: {mse_train:.2f}')
print(f'Test R^2: {r2_test:.2f}')
print(f'Test MSE: {mse_test:.2f}')


Training R^2: 0.13
Training MSE: 0.82
Test R^2: 0.21
Test MSE: 0.69


In [ ]:
# Perform 5-fold cross-validation
cv_scores = cross_val_score(model, X_selected, y, cv=5, scoring='r2')
print(f'Cross-Validation R^2 scores: {cv_scores}')
print(f'Mean CV R^2: {cv_scores.mean():.2f}')


Cross-Validation R^2 scores: [ -6.17062911 -12.83661762  -5.79341047 -37.02217444 -91.54668794]
Mean CV R^2: -30.67


In [ ]:
# Display model coefficients for selected features
coefficients = pd.DataFrame(model.coef_, selected_features, columns=['Coefficient'])
print(coefficients)


             Coefficient
EState_VSA5     0.003668
VSA_EState7     0.004384
SMR_VSA10       0.011544
PEOE_VSA7       0.002529
SlogP_VSA7      0.042699


In [ ]:
# Predict the pIC50 value for a new compound
new_smiles = 'CCO'
new_descriptors = calculate_all_descriptors(new_smiles).reindex(selected_features).values.reshape(1, -1)
predicted_pIC50 = model.predict(new_descriptors)
print(f'Predicted pIC50 for {new_smiles}: {predicted_pIC50[0]:.2f}')


Predicted pIC50 for CCO: 6.72


[10:15:34] DEPRECATION WARNING: please use MorganGenerator
[10:15:34] DEPRECATION WARNING: please use MorganGenerator
[10:15:34] DEPRECATION WARNING: please use MorganGenerator
